Есть корпус текстов. В нем есть выделенные при помощи каких-то мер устойчевые словосочетания. Надо организовать хранение и поиск текстов и словосочетаний. То есть, например, пользователь вводит сочетание, а мы ему показываем фрагменты текстов с ним (например, предложение), плюс метаинформацию по тексту.

r.set('a', 'c') - новый объект 'ключ - значение'
r.get('a') - получить значение ключа 'а'
r.mget('a', 'c') - получить значения ключей 'а' и 'с'
r.mget('a', 'c') - получить значения ключей 'а' и 'с'
r.append("a", "c") - добавить значение 'с' в ключ 'а'
r.hset(key, value, number) - key[value] = number
r.hmset(key, {value1:number1, value2:number2})
r.hget(key, vlue) > number
r.hget(key) - всё

In [4]:
!pip3 install redis

In [174]:
import redis
import json

from string import punctuation
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from nltk import tokenize
import re
import string

In [180]:
morph = MorphAnalyzer()
punct = punctuation+'«»―…“”*№–'

In [179]:
def normalize(text):
    if type(text) is str:
        words = [word.strip(punct) for word in text.lower().split()]
        words = [morph.parse(word)[0].normal_form for word in words if word]
        return ' '.join(words)
    else:
        for idx, sent in enumerate(text):
            words = [word.strip(punct) for word in sent.lower().split()]
            words = [morph.parse(word)[0].normal_form for word in words if word]
            text[idx] = ' '.join(words)
        return text

In [6]:
r = redis.StrictRedis(host='localhost', port=6379, db=0)

### Состав БД

1) хранение текстов и метаданных;
hmset('text:int', {'t_id':t_id, 'test':text, 'norm_text':norm_tex, 'title':title, 'author':author, 'year':year})
2) хранение фраз и значений;
hmset('phrase', {'p_id':p_id, 'meaning':meaning})
3) хранение соответствий фраз и текстов/предложений;
hset(where, 'p_id':'{t_id:[sents], t_id:[sents]}')
4) счётчик текстов и фраз;
hmset('maxes', {'t_max':t_id_last, 'p_max':p_id_last'}

# Положим в БД данные для экспериментов

Данные сейчас введу вручную, мелкими глупыми функциями, чтобы не заморачиваться. Аналогичные этим будут использоватья ниже, но я их буду переписывать, чтобы было красивее.

### Положим пару текстов

In [717]:
with open ('test_texts.txt', encoding = 'UTF-8') as f:
    texts = f.read()

with open ('test_texts.txt', encoding = 'UTF-8') as f:
    norm_texts = f.read()

In [720]:
texts = texts.split('\n\n')
norm_texts = norm_texts.split('\n\n')

In [725]:
norm_text = []
for n in range(len(texts)):
    texts[n] = texts[n].replace('\ufeff', '').replace('…', '.')
    texts[n] = texts[n].split('\n')
    norm_texts[n] = norm_texts[n].replace('\ufeff', '').replace('…', '.')
    norm_texts[n] = norm_texts[n].split('\n')
    texts[n][0] = tokenize.sent_tokenize(texts[n][0])
    norm_text.append(normalize(tokenize.sent_tokenize(norm_texts[n][0])))

In [726]:
texts[0]

[['― Бедные.',
  '― Настя совсем расстроилась.',
  '― Да уж, им не позавидуешь, ― согласился папа.',
  '― Но не надо вешать нос!',
  'Разве мы с вами нашего Барсика не на улице нашли?',
  'Даже с бездомными кошками случаются настоящие чудеса.',
  'Однако что же мы стоим на морозе?'],
 'Екатерина Каретникова',
 'Зимняя сказка ',
 '2011']

In [727]:
norm_text[0]

['бедный',
 'настя совсем расстроиться',
 'да уж имя не позавидовать согласиться папа',
 'но не надо вешать нос',
 'разве мы с вы наш барсик не на улица найти',
 'даже с бездомный кошка случаться настоящий чудо',
 'однако что же мы стоять на мороз']

In [238]:
texts[0][3]

'2011'

In [323]:
def fill_texts(texts, norm_text):
    for i in range(len(texts)):
        #i - t_id
        t = 'text:'+str(i)
        r.hmset(t, {'t_id':str(i), 'text':str(texts[i][0]), 'norm_text':str(norm_text[i]), 'title':texts[i][2], 
                       'author':texts[i][1], 'year':texts[i][3]})
        r.hmset('maxes', {'t_max':str(i), 'p_max':'None'})

In [324]:
fill_texts(texts, norm_text)

In [326]:
r.hget('text:0', 'title').decode('utf-8')

'Зимняя сказка '

In [327]:
r.hgetall('maxes')

{b'p_max': b'None', b't_max': b'11'}

### Положим пару фраз

In [265]:
with open ('text_phrases.txt', encoding = 'UTF-8') as f:
    phrases = f.read()

In [266]:
phrases= phrases.split('\n\n')

In [347]:
phrases

['бить баклуши\nлениться',
 'кот наплакал\nочень мало',
 'ахилесова пята\nочень мало',
 'бабье лето\nтёплые дни ранней осени',
 'бальзам на душу\nчто-то, что успокаивает тревогу',
 'битый час\nочень долго']

In [267]:
ph = []
mea = []
for p in phrases:
    ph.append(p.split('\n')[0])
    mea.append(p.split('\n')[1])

In [270]:
ph[0], mea[0]

('бить баклуши', 'лениться')

In [352]:
def put_phs(ph, mea):
    for i in range(len(ph)):
        r.hmset(ph[i], {'p_id':str(i), 'meaning':mea[i]})
    r.hmset('maxes', {'p_max':str(i)})

In [353]:
put_phs(ph, mea)

In [357]:
r.hgetall('битый час')

{b'meaning': b'\xd0\xbe\xd1\x87\xd0\xb5\xd0\xbd\xd1\x8c \xd0\xb4\xd0\xbe\xd0\xbb\xd0\xb3\xd0\xbe',
 b'p_id': b'5'}

In [359]:
r.hgetall('maxes')

{b'p_max': b'5', b't_max': b'11'}

In [655]:
#подходит для дальнейшего использования
def get_where(phrase):
    norm_phrase =  normalize(phrase)
    dic = {}
    l = int(r.hget('maxes', 't_max').decode('utf-8')) #количество текстов
    for i in range(l):
        sents = []
        t = 'text:' + str(i)
        if r.exists(t) == 1:
            norm_sents = r.hget(t, 'norm_text').decode('utf-8').strip("[]").split(', ')
            for s in range(len(norm_sents)):
                if norm_phrase in norm_sents[s]:
                    sents.append(str(s))
            if sents != []:
                dic[i] = sents
        else:
            continue
    dic = json.dumps(dic)
    if dic != '{}':
        p_id = r.hget(phrase, 'p_id').decode('utf-8')
        r.hset('where', p_id, dic)
    return

In [654]:
r.exists('text:22') ==0

True

In [797]:
for phrase in ph:
    get_where(phrase)

In [481]:
r.hgetall('where')

{b'0': b'{"10": ["0"]}',
 b'1': b'{"8": ["3"], "9": ["3"]}',
 b'3': b'{"6": ["4"], "7": ["3"]}',
 b'4': b'{"4": ["3"], "5": ["3"]}'}

## Поиск по выражению

In [564]:
def print_info(all_info):
    print(all_info[0] + '\n')
    print('Название текста: ' + ''.join(all_info[1]))
    print('Автор текста: ' + all_info[2])
    print('Год создания текста: ' + all_info[3] + '\n')
    print('~~~')
    return

In [581]:
def get_info(t_id):
    all_info = []
    t_id = 'text:'+t_id
    te = r.hget('text:1', 'text').decode('utf-8').strip("[']").replace("', '", " ")
    ti = r.hget(t_id, 'title').decode('utf-8')
    au = r.hget(t_id, 'author').decode('utf-8')
    ye = r.hget(t_id, 'year').decode('utf-8')
    all_info.extend([te, ti, au, ye])
    print_info(all_info)
    return

In [582]:
def search_in_base(phrase):
    if r.exists(phrase) is 1:
        meaning = r.hget(phrase, 'meaning').decode('utf-8')
        p_id = r.hget(phrase, 'p_id').decode('utf-8')
        print
        print('Значение: ' + meaning + '\n')
        if r.hexists('where', p_id) is True:
            d = json.loads(r.hget('where', p_id).decode('utf-8'))
            where_p = list(d.keys())
            for t_id in where_p:
                get_info(t_id)
        else:
            print('К сожалению, примеров нет.')
    else:
        print('Такого выражения в базе нет.')
    return

In [585]:
search_in_base('кот наплакал')

Значение: очень мало

― Вообще ― многое поздно. Я не ответил, потому что это замечание не предполагало ответа. Собственно, почти всё было уже ясно. ― Не вешайте нос, прапорщик, ― произнёс я совершенно механически. Так военный хирург в приёмном покое раз за разом привычно говорит своим искалеченным пациентам что-нибудь вроде: «Мы ещё с вами водки выпьем, лейтенант» . ― Я не вешаю, ― живо возразил Дыбовский. ― Просто надо же реалистично смотреть на вещи.

Название текста: Многоточие сборки
Автор текста: Ю. И. Андреева
Год создания текста: 2009

~~~
― Вообще ― многое поздно. Я не ответил, потому что это замечание не предполагало ответа. Собственно, почти всё было уже ясно. ― Не вешайте нос, прапорщик, ― произнёс я совершенно механически. Так военный хирург в приёмном покое раз за разом привычно говорит своим искалеченным пациентам что-нибудь вроде: «Мы ещё с вами водки выпьем, лейтенант» . ― Я не вешаю, ― живо возразил Дыбовский. ― Просто надо же реалистично смотреть на вещи.

Название те

In [584]:
search_in_base('ахилесова пята')

Значение: очень мало

К сожалению, примеров нет.


In [586]:
search_in_base('второе дыхание')

Такого выражения в базе нет.


## Добавление фразы

In [608]:
def add_phrase(phrase, meaning):
    if r.exists(phrase) is 1:
        print('Такое выражение уже есть в базе.')
    else:
        new_id = str(int(r.hget('maxes', 'p_max').decode('utf-8')) + 1)
        r.hmset('maxes', {'p_max':new_id})
        r.hmset(phrase, {'p_id':new_id, 'meaning':meaning})
        get_where(phrase)
        print('Выражение успешно добавлено!')
    return

In [602]:
my_phrase = 'бить баклуши'
mea = 'лениться'
add_phrase(my_phrase, mea)

Такое выражение уже есть в базе.


In [695]:
my_phrase = 'белая ворона'
mea = 'непохожий на других'
add_phrase(my_phrase, mea)

Выражение успешно добавлено!


In [615]:
r.hgetall('белая ворона')

{b'meaning': b'\xd0\xbd\xd0\xb5\xd0\xbf\xd0\xbe\xd1\x85\xd0\xbe\xd0\xb6\xd0\xb8\xd0\xb9 \xd0\xbd\xd0\xb0 \xd0\xb4\xd1\x80\xd1\x83\xd0\xb3\xd0\xb8\xd1\x85',
 b'p_id': b'8'}

## Удаление фразы

In [694]:
def delete_phrase(phrase):
    i_d = r.hget(phrase, 'p_id')
    r.hdel('where', i_d)
    r.delete(phrase)
    print('Выражение удалено из базы.')
    return

In [696]:
r.exists('белая ворона')

1

In [697]:
delete_phrase('белая ворона')

Выражение удалено из базы.


In [698]:
r.exists('белая ворона')

0

## Добавление текста

In [701]:
with open ('new_text.txt', encoding = 'UTF-8') as f:
    text = f.read()
    
auth = 'Юрий Трифонов'
title = 'Предварительные итоги'
year = '1970'

In [801]:
def add_sents(all_phrases, t_id):
    t = 'text:' + str(t_id)
    norm_sents = r.hget(t, 'norm_text').decode('utf-8').strip("[]").split(', ')
    for phrase in all_phrases:
        sents = []
        norm_phrase =  normalize(phrase)
        p_id = r.hget(phrase, 'p_id').decode('utf-8')
        if r.hexists('where', p_id):
            dict_where = json.loads(r.hget('where', p_id))
        else:
            dict_where = {}
        for s in range(len(norm_sents)):
            if norm_phrase in norm_sents[s]:
                sents.append(str(s))
        if sents != []:
            dict_where[t_id] = sents
        dict_where = json.dumps(dict_where)
        if dict_where != '{}':
            r.hset('where', p_id, dict_where)
    return

In [824]:
def get_all_phrases():
    all_phrases = []
    for a in r.keys():
        a = a.decode('utf-8')
        if 'text' not in a and 'maxes'not in a and 'where' not in a:
            all_phrases.append(a)
    return(all_phrases)

In [739]:
def add_text(text, auth, title, year):
    text = text.replace('…', '.').strip('\ufeff\n')
    text = tokenize.sent_tokenize(text)
    norm_text = normalize(text)
    new_id = str(int(r.hget('maxes', 't_max').decode('utf-8')) + 1)
    t = 'text:'+new_id
    r.hmset(t, {'t_id':new_id, 'text':str(text), 'norm_text':str(norm_text), 'title':title, 'author':auth, 'year':year})
    all_phrases = det_all_phrases()
    add_sents(all_phrases, new_id)
    r.hmset('maxes', {'t_max':new_id})
    print('Новый текст добавлен.')
    return

In [815]:
r.hgetall('where')

{b'0': b'{"10": ["0"], "11": ["3"]}',
 b'1': b'{"8": ["3"], "9": ["3"]}',
 b'3': b'{"6": ["4"], "7": ["3"]}',
 b'4': b'{"4": ["3"], "5": ["3"]}'}

In [847]:
add_text(text, auth, title, year)

Новый текст добавлен.


In [817]:
r.hgetall('where')

{b'0': b'{"10": ["0"], "11": ["3"]}',
 b'1': b'{"8": ["3"], "9": ["3"]}',
 b'3': b'{"6": ["4"], "7": ["3"]}',
 b'4': b'{"4": ["3"], "5": ["3"]}',
 b'5': b'{"12": ["0"]}'}

## Удаление текста

In [839]:
def get_one_text(title):
    for a in r.keys():
        a = a.decode('utf-8')
        if 'text' in a and r.hget(a, 'title').decode('utf-8') == title:
            return(a)

In [869]:
lol = {"hello":"gdbye"}
a = lol.pop("hello")
lol == {}

True

In [870]:
def delete_where(t_id):
    all_phrases = get_all_phrases()
    for phrase in all_phrases:
        p_id = r.hget(phrase, 'p_id').decode('utf-8')
        if r.hexists('where', p_id):
            dict_where = json.loads(r.hget('where', p_id))
            try:
                a = dict_where.pop(t_id)
            except:
                continue
            if dict_where != {}:
                dict_where = json.dumps(dict_where)
                r.hset('where', p_id, dict_where)
            else:
                r.hdel('where', p_id)
    return

In [851]:
def delete_text(title):
    text = get_one_text(title)
    t_id = r.hget(text, 't_id')
    delete_where(t_id)
    r.delete(text)
    print('Текст удалён.')
    return

In [818]:
r.hgetall('where')

{b'0': b'{"10": ["0"], "11": ["3"]}',
 b'1': b'{"8": ["3"], "9": ["3"]}',
 b'3': b'{"6": ["4"], "7": ["3"]}',
 b'4': b'{"4": ["3"], "5": ["3"]}',
 b'5': b'{"12": ["0"]}'}

In [854]:
delete_text(title)

Текст удалён.


In [873]:
r.hgetall('where')

{b'0': b'{"10": ["0"], "11": ["3"]}',
 b'1': b'{"8": ["3"], "9": ["3"]}',
 b'3': b'{"6": ["4"], "7": ["3"]}',
 b'4': b'{"4": ["3"], "5": ["3"]}'}

## Редактирование текста

In [877]:
def get_typ_text(typ, data):
    for a in r.keys():
        a = a.decode('utf-8')
        if 'text' in a and r.hget(a, typ).decode('utf-8') == data:
            return(a)

In [881]:
def change(typ, data, new_data):
    text = get_typ_text(typ, data)
    r.hmset(text, {typ:new_data})
    return

In [875]:
r.hget('text:11', 'title').decode('utf-8')

'Повесть о жизни'

In [882]:
change('title', 'Повесть о жизни', 'Повесть о смерти')

In [883]:
r.hget('text:11', 'title').decode('utf-8')

'Повесть о смерти'